In [33]:
from client import NineGagAPIClient

In [ ]:
import requests

url = "https://9gag.com/v1/feed-posts/type/home"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Referer": "https://9gag.com/",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
    "Cookie": "____ri=6854; ____lo=DE; sign_up_referer=https%3A%2F%2F9gag.com%2F; ts1=9f9ecd24c7dd0e753be3d450d0c381f6ae9fab00; OptanonConsent=isGpcEnabled=0&datestamp=Tue+Jan+07+2025+16%3A04%3A54+GMT%2B0100+(Central+European+Standard+Time)&version=202408.1.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=d6bd00d0-d975-41a3-8a52-149f70856088&interactionCount=0&isAnonUser=1&landingPath=NotLandingPage&groups=C0004%3A1%2CC0002%3A1%2CC0003%3A1%2CC0001%3A1&AwaitingReconsent=false&geolocation=%3B; OptanonAlertBoxClosed=2025-01-07T15:04:54.894Z"
}
response = requests.get(url, headers=headers)
print(response.text)  # Inspect the content

if response.status_code != 200:
    print("Failed to fetch data:", response.status_code)

print(response.json())  # Inspect the structure

In [12]:
import json
import pandas as pd
import cloudscraper
from datetime import datetime

def page_statistic_df():
    # Create a CloudScraper instance
    scraper = cloudscraper.create_scraper()
    url_9gag = "https://9gag.com/v1/feed-posts/type/home"
    response = scraper.get(url=url_9gag)
    response_json = json.loads(response.text)

    keys_to_keep = ['id', 'title', 'description', 'type', 'nsfw', 'upVoteCount', 'downVoteCount', 'creationTs' ]
    all_filtered_data = []
    for data_per_post in response_json["data"]["posts"]:
        filtered_data = {key: data_per_post[key] for key in keys_to_keep}
        all_filtered_data.append(filtered_data)

    # Convert to a DataFrame
    df = pd.DataFrame(all_filtered_data)
    df['creationTs'] = pd.to_datetime(df['creationTs'], unit='s')
    return df

In [13]:
some = page_statistic_df()

In [ ]:
import cloudscraper
import pandas as pd

def fetch_comments(post_id, count=10, sort_type="hot", next_token=None):
    """
    Fetch comments from a 9GAG post using the observed API.
    
    :param post_id: The ID of the 9GAG post (e.g., 'aLnngEz').
    :param count: Number of comments to fetch per request (default 10).
    :param sort_type: Sorting type ('hot', 'fresh', etc.).
    :param next_token: Token for pagination (default None).
    :return: List of comments with relevant information.
    """
    scraper = cloudscraper.create_scraper()
    base_url = "https://comment-cdn.9gag.com/v2/cacheable/comment-list.json"

    # Query parameters
    params = {
        "appId": "a_dd8f2b7d304a10edaf6f29517ea0ca4100a43d1b",
        "count": count,
        "type": sort_type,
        "url": f"http://9gag.com/gag/{post_id}",
        "origin": "https://9gag.com",
    }
    if next_token:
        params["next"] = next_token

    # Make the request
    response = scraper.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        payload = data.get("payload", [])
        comments = payload.get("comments", [])
        next_token = payload.get("next")
        print(data)
        # Extract relevant data
        comment_list = [
            {
                "comment_id": comment.get("commentId"),
                "text": comment.get("text"),
                "timestamp": comment.get("timestamp"),
            }
            for comment in comments if comment.get('type')  == 'text'
        ]
        df = pd.DataFrame(comment_list)
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
        return df, next_token
    else:
        print(f"Error {response.status_code}: {response.text}")
        return [], None

# Example Usage
post_id = "aLnngEz"  # Replace with the actual post ID
df, next_token = fetch_comments(post_id)



In [48]:
import cloudscraper
import pandas as pd

def fetch_comments1(post_id, count=10, sort_type="hot", next_token=None):
    """
    Fetch comments from a 9GAG post using the observed API.
    
    :param post_id: The ID of the 9GAG post (e.g., 'aLnngEz').
    :param count: Number of comments to fetch per request (default 10).
    :param sort_type: Sorting type ('hot', 'fresh', etc.).
    :param next_token: Token for pagination (default None).
    :return: DataFrame of comments and the next pagination token.
    """
    scraper = cloudscraper.create_scraper()
    base_url = "https://comment-cdn.9gag.com/v2/cacheable/comment-list.json"

    # Query parameters
    params = {
        "appId": "a_dd8f2b7d304a10edaf6f29517ea0ca4100a43d1b",
        "count": count,
        "type": sort_type,
        "url": f"http://9gag.com/gag/{post_id}",
        "origin": "https://9gag.com",
    }
    if next_token:
        params["next"] = next_token  # Include the next_token for pagination

    # Make the request
    response = scraper.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        payload = data.get("payload", {})
        comments = payload.get("comments", [])
        next_token = payload.get("next")  # Retrieve the next token for pagination

        # Extract relevant data
        comment_list = [
            {
                "comment_id": comment.get("commentId"),
                "text": comment.get("text"),
                "timestamp": comment.get("timestamp"),
            }
            for comment in comments if comment.get("type") == "text"
        ]

        # Create a DataFrame and format the timestamp
        df = pd.DataFrame(comment_list)
        if not df.empty:
            df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")
        return df, next_token
    else:
        print(f"Error {response.status_code}: {response.text}")
        return pd.DataFrame(), None

# Fetch all comments with pagination
def fetch_all_comments(post_id, count=10, sort_type="hot"):
    """
    Fetch all comments for a given 9GAG post, handling pagination.
    
    :param post_id: The ID of the 9GAG post.
    :param count: Number of comments per request.
    :param sort_type: Sorting type for comments.
    :return: DataFrame of all comments.
    """
    all_comments = pd.DataFrame()
    next_token = None

    while True:
        df, next_token = fetch_comments(post_id, count, sort_type, next_token)
        all_comments = pd.concat([all_comments, df], ignore_index=True)

        if not next_token:  # No more pages
            break

    return all_comments



In [51]:
post_id = "aLnngEz"  # Replace with the actual post ID

df, next_token = fetch_comments1(post_id)

In [ ]:
next_token

In [53]:
df1, next_token1 = fetch_comments1(post_id,next_token=next_token)

In [ ]:
next_token1

In [ ]:
df

In [ ]:
df1